The code is taken from: https://github.com/azure/azureml-examples/blob/main/sdk/python/jobs/automl-standalone-jobs/automl-regression-task-hardware-performance/automl-regression-task-hardware-performance.ipynb 

# **Change the job_name name according to your setup**

# Download the Best Model and Related Artifacts

We assume that you are using the same environment that was used in the previous code.

## Connect to your workspace

In [14]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [15]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## Get the best model

Get the Automl parent job

In [16]:
from mlflow.tracking.client import MlflowClient

mlflow_client = MlflowClient()

# **Change the job_name name according to your setup**

### You can automated this process too.

In [17]:
# Take the job name from Jobs, their are automated way also
job_name = 'patient_stone_f7q7hjzq2j'


# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)


print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.9907829151390034,
 'AUC_micro': 0.9918133000000001,
 'AUC_weighted': 0.9907829151390034,
 'accuracy': 0.9559999999999998,
 'average_precision_score_macro': 0.9888143846430495,
 'average_precision_score_micro': 0.9918785155939377,
 'average_precision_score_weighted': 0.9909895133351162,
 'balanced_accuracy': 0.9489702204208188,
 'f1_score_macro': 0.9503549395397359,
 'f1_score_micro': 0.9559999999999998,
 'f1_score_weighted': 0.9559279562980892,
 'log_loss': 0.15783529277634573,
 'matthews_correlation': 0.9008131075598756,
 'norm_macro_recall': 0.8979404408416375,
 'precision_score_macro': 0.9518543009366212,
 'precision_score_micro': 0.9559999999999998,
 'precision_score_weighted': 0.955947856676608,
 'recall_score_macro': 0.9489702204208188,
 'recall_score_micro': 0.9559999999999998,
 'recall_score_weighted': 0.9559999999999998,
 'weighted_accuracy': 0.9616123964291521}, params={}, tags={'automl_best_child_run_id': 'patient_sto

In [18]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

{'model_explain_run': 'best_run', 'pipeline_id_000': '5dfac790c5c209f98a1da2dc1c7fb76f0397324f;c7af0367625be6ac5c2fecbfc72ed444cb7a2111;799d2168db11fc19b9e1c6c1df62f8981ad39fe9;44b5d2d23120dfcf96a7dc6389a293d17a966b1b;3735210984ea10097d5c91905cf6300ac278cd89;5304e705d6f401cb07f85bcf0ca59ac1f84dfcf5;294650ec23bc3d067a4972de7ceb12f6e97260fa;79ebb3b8c385e3190e3dd4c1fa28b74730528045;589334a57221e0720a95cf217c065115a7b841d1;5a8eb9d9f67b1dd7e9e4062585daa2417ea78989;35b8e2db3c5f6e714072e1be9f0210fe48426975;797e3e972ad02d947be4c64d2b06b775b6df7a49;3fd2e115482e0c7b92aa3297688aefb6c3aa6fb8;1dfdf370ba770047b14bfa59c40920815defcdab;17f87d449bb43aaf3cdae4b7eb6dd6e7fbdaf343;dc68fbd594530b5d52812dd9b3ffa3d6d8ba625e;2032292dd602836ed754a266c771ac100c4c80dc;fedbb4995d6de479c513576119965d71276f03d1;e478544eb8734c07a1943f5bdf4e6605be6943df;1b9ea9b05e1aba0cfbaa30fae26f336a29920979;3ef9040509da05037cd402691a100d4b3dfeb7c4;8a629c3253eba80dbb72f47024d32f0ad25a75b4;13a209bfd4b2554ade8a6975e6538fc9cf736b47;027

## Get the AutoMl best child run 

In [19]:
# Get the best model's child run
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)


best_run = mlflow_client.get_run(best_child_run_id)


print("Best child run: ")
print(best_run)

Found best child run id:  patient_stone_f7q7hjzq2j_47
Best child run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.9907829151390034,
 'AUC_micro': 0.9918133000000001,
 'AUC_weighted': 0.9907829151390034,
 'accuracy': 0.9559999999999998,
 'average_precision_score_macro': 0.9888143846430495,
 'average_precision_score_micro': 0.9918785155939377,
 'average_precision_score_weighted': 0.9909895133351162,
 'balanced_accuracy': 0.9489702204208188,
 'f1_score_macro': 0.9503549395397359,
 'f1_score_micro': 0.9559999999999998,
 'f1_score_weighted': 0.9559279562980892,
 'log_loss': 0.15783529277634573,
 'matthews_correlation': 0.9008131075598756,
 'norm_macro_recall': 0.8979404408416375,
 'precision_score_macro': 0.9518543009366212,
 'precision_score_micro': 0.9559999999999998,
 'precision_score_weighted': 0.955947856676608,
 'recall_score_macro': 0.9489702204208188,
 'recall_score_micro': 0.9559999999999998,
 'recall_score_weighted': 0.9559999999999998,
 'weighted_accuracy': 0.9616123964291521},

## Get best model run's validation metrics

In [20]:
import pandas as pd

pd.DataFrame(best_run.data.metrics, index=[0]).T

,0
AUC_macro,0.990783
precision_score_micro,0.956000
f1_score_macro,0.950355
matthews_correlation,0.900813
average_precision_score_macro,0.988814
f1_score_weighted,0.955928
log_loss,0.157835
weighted_accuracy,0.961612
norm_macro_recall,0.897940
recall_score_micro,0.956000


## Download the best model

In [21]:
import os

# Create local folder
local_dir = "./artifact_downloads/"
os.makedirs(local_dir, exist_ok=True)

In [22]:
from mlflow.artifacts import download_artifacts

# Download run's artifacts/outputs
local_path = download_artifacts(
    run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Artifacts downloaded in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/tj-compute-instance/code/Users/taseen.junaid/Azure-AutoML-Custom-Deployment/artifact_downloads/outputs
Artifacts: ['conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'generated_code', 'internal_cross_validated_models.pkl', 'mlflow-model', 'model.onnx', 'model.pkl', 'model_onnx.json', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_pbi_v_1_0_0.py', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']


## View featurization summary

In [23]:
import json

# Render the JSON as a pandas DataFrame
with open(os.path.join(local_path, "featurization_summary.json"), "r") as f:
    records = json.load(f)
fs = pd.DataFrame.from_records(records)


# View a summary of the featurization
fs[
    [
        "RawFeatureName",
        "TypeDetected",
        "Dropped",
        "EngineeredFeatureCount",
        "Transformations",
    ]
]


,RawFeatureName,TypeDetected,Dropped,EngineeredFeatureCount,Transformations
0,PatientID,Numeric,No,1,[MeanImputer]
1,PlasmaGlucose,Numeric,No,1,[MeanImputer]
2,DiastolicBloodPressure,Numeric,No,1,[MeanImputer]
3,TricepsThickness,Numeric,No,1,[MeanImputer]
4,SerumInsulin,Numeric,No,1,[MeanImputer]
5,BMI,Numeric,No,1,[MeanImputer]
6,DiabetesPedigree,Numeric,No,1,[MeanImputer]
7,Age,Numeric,No,1,[MeanImputer]
8,Pregnancies,Categorical,No,15,[StringCast-CharGramCountVectorizer]
